In [1]:
with open('genes_2510.txt', 'r') as file:
    # Read the contents of the file into a list using readlines()
    genes_2510 = list(map(lambda x: x[:-1], file.readlines()))
print(len(genes_2510))
print(genes_2510[:5])

2510
['UBB', 'UBC', 'UBA52', 'RPS27A', 'UBA1']


In [2]:
import pandas as pd

drug_response_df = pd.read_csv("../0208/cell_drug_auc_final_1111.csv")
print(drug_response_df.shape)
drug_response_df.head()

(692, 386)


,ARXSPAN_ID,JW-7-24-1,KIN001-260,NSC-87877,GNE-317,NAVITOCLAX,PLX-4720,ERK5-IN-1,VX-11E,TGX-221,...,BIX02189,ISPINESIB MESYLATE,KIN001-135,KIN001-206,KIN001-236,KIN001-266,LUMINESPIB,NUTLIN-3A,SGC0946,SL 0101-1
0,ACH-000001,0.778432,0.951321,0.840287,-99999.000000,-99999.000000,0.936410,0.891908,0.402122,0.659600,...,0.982272,0.733492,0.072277,0.983536,-99999.000000,0.651746,0.982084,0.893777,0.983861,0.933035
1,ACH-000002,0.788327,0.874760,0.760137,0.905754,0.930764,0.991068,0.278288,0.972905,0.558774,...,0.899628,0.604228,0.046061,0.918265,0.865628,-99999.000000,0.985921,0.839175,0.988639,0.533204
2,ACH-000004,0.737630,0.938733,0.835441,0.808964,0.966133,0.983552,-99999.000000,0.993912,0.596027,...,0.965245,0.782295,0.366912,0.844979,0.761323,-99999.000000,0.980945,0.946694,0.992605,0.761956
3,ACH-000006,0.176396,0.571569,0.581729,0.585938,0.958510,0.935574,0.600428,-99999.000000,0.596056,...,0.834158,0.605360,0.124930,0.813384,0.782187,-99999.000000,0.971058,0.871277,-99999.000000,0.495020
4,ACH-000007,0.569751,0.755449,0.954924,0.939587,0.965638,0.932666,-99999.000000,0.956481,0.838692,...,0.927519,0.812955,0.719780,0.815099,0.795736,0.860768,0.983317,0.888067,0.987564,0.787948


In [3]:
import numpy as np

# define column names
columns = list(genes_2510)

# define row index names
index_names = drug_response_df.ARXSPAN_ID.values

# create empty dataframe with column names, and row index
data = np.zeros((len(index_names), len(columns))) # initialize all value to be 0
cnv_df = pd.DataFrame(data, columns=columns, index=index_names)
cnv_df.shape

(692, 2510)

In [4]:
### CNV data
CCLE_cnv = pd.read_csv('CCLE_segment_cn_21Q3.csv', low_memory=False, index_col=0)
CCLE_cnv.head()

,Chromosome,Start,End,Segment_Mean,Num_Probes,Status,Source
DepMap_ID,,,,,,,
ACH-000001,1,0,6275858,1.762136,5148,+,Broad WGS
ACH-000001,1,6275859,7079442,2.490965,792,+,Broad WGS
ACH-000001,1,7079443,7502035,1.849405,417,+,Broad WGS
ACH-000001,1,7502035,14866457,1.778401,6664,+,Broad WGS
ACH-000001,1,14866458,15395313,2.450038,514,+,Broad WGS


In [5]:
import requests
import json

def gene_info_query(gene):
    # set up the URL and parameters for the API request
    url = "http://rest.ensembl.org/lookup/symbol/homo_sapiens/"
    gene_symbol = gene
    params = {
        "content-type": "application/json",
        "expand": "1"
    }

    # send the API request and get the response
    response = requests.get(url + gene_symbol, headers=params)

    # check if the response is successful
    if response.status_code == requests.codes.ok:
        # parse the response as JSON
        data = json.loads(response.text)
        # extract the location information from the response
        location = data["seq_region_name"] + ":" + str(data["start"]) + "-" + str(data["end"])
        print("Location of " + gene_symbol + " gene:", location)
        start_pos = location.split(":")[1].split("-")[0]
        end_pos = location.split(":")[1].split("-")[1]
        chromosome = location.split(":")[0]
        return {"start": int(start_pos), "end": int(end_pos), "chromosome": chromosome}
    else:
        # handle the error
        response.raise_for_status()

In [6]:
meta_gene_info_dict = {}
missing_gene_lst = []
for gene in genes_2510:
    try:
        meta_gene_info_dict[gene] = gene_info_query(gene)
    except:
        missing_gene_lst.append(gene)
print(len(meta_gene_info_dict))

Location of UBB gene: 17:16380798-16382745
Location of UBC gene: 12:124911604-124917368
Location of UBA52 gene: 19:18571730-18577550
Location of RPS27A gene: 2:55231903-55235853
Location of UBA1 gene: X:47190861-47215128
Location of SAE1 gene: 19:47113274-47210636
Location of UBA2 gene: 19:34428352-34471251
Location of UBA3 gene: 3:69054730-69080408
Location of UBA7 gene: 3:49805209-49813953
Location of UBA6 gene: 4:67612652-67701155
Location of UBE2A gene: X:119474802-119591083
Location of UBE2B gene: 5:134371184-134392108
Location of UBE2C gene: 20:45812576-45816957
Location of UBE2D4 gene: 7:43926436-43956136
Location of UBE2D1 gene: 10:58334979-58370751
Location of UBE2D2 gene: 5:139526431-139628434
Location of UBE2D3 gene: 4:102794383-102868896
Location of UBE2E3 gene: 2:180967248-181076585
Location of UBE2E1 gene: 3:23805955-23891640
Location of UBE2E2 gene: 3:23203020-23591794
Location of UBE2F gene: 2:237966827-238042782
Location of UBE2G1 gene: 17:4269259-4366675
Location of U

Location of RGMB gene: 5:98768650-98798643
Location of AMHR2 gene: 12:53423855-53431672
Location of TGFBR1 gene: 9:99104038-99154192
Location of TGFBR2 gene: 3:30606601-30694142
Location of ACVR1B gene: 12:51951699-51997078
Location of ACVR2B gene: 3:38453890-38493142
Location of ACVR1C gene: 2:157526767-157628864
Location of BAMBI gene: 10:28677510-28682932
Location of SMAD1 gene: 4:145481194-145559176
Location of SMAD5 gene: 5:136132845-136188747
Location of SMAD9 gene: 13:36844831-36920765
Location of SMAD2 gene: 18:47808957-47931146
Location of SMAD3 gene: 15:67063763-67195173
Location of SMAD4 gene: 18:51028394-51085045
Location of SMAD6 gene: 15:66702236-66782849
Location of SMAD7 gene: 18:48919853-48950965
Location of ZFYVE9 gene: 1:52142089-52348671
Location of ZFYVE16 gene: 5:80408013-80483379
Location of HAMP gene: 19:35280716-35285143
Location of ID1 gene: 20:31605283-31606515
Location of ID2 gene: 2:8678845-8684461
Location of ID3 gene: 1:23557926-23559501
Location of ID4 g

Location of CACNA1D gene: 3:53328963-53813733
Location of CACNA1E gene: 1:181317690-181813262
Location of CACNA1F gene: X:49205063-49233371
Location of CACNA1G gene: 17:50560715-50627474
Location of CACNA1H gene: 16:1153106-1221771
Location of CACNA1I gene: 22:39570753-39689735
Location of CACNA1S gene: 1:201039512-201112451
Location of CACNA2D1 gene: 7:81946444-82443956
Location of CACNA2D2 gene: 3:50362613-50504244
Location of CACNA2D3 gene: 3:54122547-55074557
Location of CACNA2D4 gene: 12:1791963-1918666
Location of CACNB1 gene: 17:39173453-39197702
Location of CACNB2 gene: 10:18140424-18543557
Location of CACNB3 gene: 12:48813794-48828941
Location of CACNB4 gene: 2:151832771-152099167
Location of CACNG1 gene: 17:67044554-67056797
Location of CACNG2 gene: 22:36560857-36703752
Location of CACNG3 gene: 16:24256335-24362412
Location of CACNG4 gene: 17:66964707-67033398
Location of CACNG5 gene: 17:66835117-66894751
Location of CACNG6 gene: 19:53991637-54012666
Location of CACNG7 gene: 

Location of MAP3K5 gene: 6:136557046-136793097
Location of MAP3K7 gene: 6:90513573-90587086
Location of MAP3K4 gene: 6:160991727-161117385
Location of TAOK2 gene: 16:29973868-29992261
Location of TAOK3 gene: 12:118149801-118372907
Location of TAOK1 gene: 17:29390363-29551903
Location of MAP2K4 gene: 17:12020829-12143830
Location of MAP2K7 gene: 19:7903843-7914478
Location of MAP2K3 gene: 17:21284672-21315232
Location of MAP2K6 gene: 17:69414697-69553865
Location of MAPK8IP1 gene: 11:45885651-45906465
Location of MAPK8IP2 gene: 22:50600793-50613981
Location of MAPK8IP3 gene: 16:1706166-1770351
Location of FLNA gene: X:154348524-154374634
Location of FLNC gene: 7:128830406-128859274
Location of FLNB gene: 3:58008398-58172251
Location of CRK gene: 17:1420689-1463162
Location of CRKL gene: 22:20917407-20953747
Location of ARRB1 gene: 11:75260122-75351705
Location of ARRB2 gene: 17:4710596-4721499
Location of MAPK8 gene: 10:48306639-48439360
Location of MAPK10 gene: 4:85990007-86594625
Loca

Location of RNF152 gene: 18:61808067-61894247
Location of RPTOR gene: 17:80544819-80966371
Location of AKT1S1 gene: 19:49869033-49878459
Location of MTOR gene: 1:11106535-11262556
Location of DEPTOR gene: 8:119873717-120050918
Location of MLST8 gene: 16:2204248-2209453
Location of TELO2 gene: 16:1493344-1510457
Location of TTI1 gene: 20:37983007-38033461
Location of CLIP1 gene: 12:122271432-122422669
Location of GRB10 gene: 7:50590063-50793462
Location of LPIN1 gene: 2:11677595-11827409
Location of LPIN3 gene: 20:41340821-41360582
Location of LPIN2 gene: 18:2885296-3013144
Location of ULK1 gene: 12:131894622-131923150
Location of ULK2 gene: 17:19770829-19867936
Location of EIF4EBP1 gene: 8:38030534-38060365
Location of EIF4E gene: 4:98879276-98929133
Location of EIF4E2 gene: 2:232550659-232583644
Location of EIF4E1B gene: 5:176630618-176646644
Location of EIF4B gene: 12:53006282-53042215
Location of RPS6 gene: 9:19375715-19380236
Location of STRADA gene: 17:63682336-63741986
Location o

...

Location of PLA2G12A gene: 4:109709989-109730070
Location of PLA2G12B gene: 10:72934762-72954806
Location of PLA2G1B gene: 12:120322115-120327779
Location of PLA2G5 gene: 1:20028179-20091911
Location of PLA2G2A gene: 1:19975431-19980416
Location of PLA2G2C gene: 1:20161253-20186518
Location of PLA2G6 gene: 22:38111495-38214778
Location of PLAAT3 gene: 11:63573195-63616883
Location of ETS1 gene: 11:128458761-128587558
Location of ETS2 gene: 21:38805183-38824955
Location of BRAP gene: 12:111642146-111685956
Location of KSR1 gene: 17:27456448-27626438
Location of KSR2 gene: 12:117453012-117968990
Location of RAPGEF5 gene: 7:22118236-22357154
Location of RALGDS gene: 9:133097720-133149334
Location of RGL1 gene: 1:183636085-183928532
Location of RGL2 gene: 6:33291654-33298942
Location of RALA gene: 7:39623565-39708120
Location of RALB gene: 2:120240064-120294710
Location of EXOC2 gene: 6:485154-693139
Location of TBK1 gene: 12:64452090-64502114
Location of REL gene: 2:60881491-60931612
Loca

Location of PPIF gene: 10:79347469-79355334
Location of TNNC1 gene: 3:52451100-52454041
Location of TNNC2 gene: 20:45823214-45833745
Location of PHKG1 gene: 7:56080283-56092996
Location of PHKG2 gene: 16:30748293-30761176
Location of PHKB gene: 16:47461123-47701523
Location of PHKA2 gene: X:18892298-18984114
Location of PHKA1 gene: X:72578814-72714319
Location of CAMK1D gene: 10:12349547-12835545
Location of CAMK1G gene: 1:209583714-209613939
Location of CAMK1 gene: 3:9757347-9769992
Location of CAMK4 gene: 5:111223653-111494886
Location of NOS1 gene: 12:117208142-117452170
Location of NOS2 gene: 17:27756766-27800529
Location of PDE1A gene: 2:182139968-182523359
Location of PDE1B gene: 12:54549601-54579239
Location of PDE1C gene: 7:31751179-32428131
Location of ITPKB gene: 1:226631690-226739323
Location of ITPKA gene: 15:41493393-41503551
Location of ITPKC gene: 19:40717112-40740860
Location of TLR4 gene: 9:117704175-117724735
Location of HIF1A gene: 14:61695513-61748259
Location of EG

Location of ADORA1 gene: 1:203090654-203167405
Location of NPY1R gene: 4:163323962-163344832
Location of SSTR1 gene: 14:38207904-38213067
Location of SSTR2 gene: 17:73165010-73176633
Location of SSTR5 gene: 16:1072747-1081454
Location of HCAR1 gene: 12:122726076-122730844
Location of HCAR2 gene: 12:122701293-122703357
Location of HCAR3 gene: 12:122714756-122716811
Location of FFAR2 gene: 19:35443907-35451767
Location of SUCNR1 gene: 3:151873643-151884619
Location of GHSR gene: 3:172443291-172448456
Location of ADCY10 gene: 1:167809386-167914215
Location of HCN2 gene: 19:589881-617159
Location of HCN4 gene: 15:73319859-73368958
Location of CNGA1 gene: 4:47935977-48016681
Location of CNGA2 gene: X:151734746-151745564
Location of CNGA3 gene: 2:98346188-98398601
Location of CNGA4 gene: 11:6234765-6244479
Location of CNGB1 gene: 16:57882340-57971128
Location of CNGB3 gene: 8:86553977-86743675
Location of ABCC4 gene: 13:95019835-95301475
Location of RAPGEF3 gene: 12:47734363-47771040
Locatio

Location of THBS2 gene: 6:169215780-169254050
Location of THBS3 gene: 1:155195588-155209051
Location of THBS4 gene: 5:79991311-80083287
Location of SPP1 gene: 4:87975667-87983532
Location of VTN gene: 17:28367284-28373091
Location of TNC gene: 9:115019575-115118207
Location of TNN gene: 1:175067833-175148075
Location of TNR gene: 1:175315194-175743616
Location of TNXB gene: 6:32041153-32115334
Location of VWF gene: 12:5948877-6124770
Location of IBSP gene: 4:87799554-87812435
Location of LPAR3 gene: 1:84811602-84893206
Location of LPAR6 gene: 13:48389567-48444704
Location of PKN1 gene: 19:14433053-14471867
Location of PKN2 gene: 1:88684222-88836255
Location of PKN3 gene: 9:128702503-128720916
Location of SGK2 gene: 20:43558968-43588237
Location of SGK3 gene: 8:66712734-66862022
Location of MAGI1 gene: 3:65353525-66038918
Location of MAGI2 gene: 7:78017055-79453667
Location of THEM4 gene: 1:151870866-151909637
Location of PPP2R2A gene: 8:26291508-26372680
Location of PPP2R2B gene: 5:146

Location of BCL6 gene: 3:187721377-187745725
Location of BCOR gene: X:40049815-40177329
Location of BCORL1 gene: X:129981107-130058071
Location of BCR gene: 22:23179704-23318037
Location of BRD4 gene: 19:15235519-15332545
Location of BTG1 gene: 12:92140278-92145846
Location of BTG2 gene: 1:203305491-203309602
Location of EMSY gene: 11:76444923-76553031
Location of CALR gene: 19:12938578-12944489
Location of CBFB gene: 16:67028984-67101058
Location of CD74 gene: 5:150401637-150412969
Location of CDC73 gene: 1:193121983-193254815
Location of CDK12 gene: 17:39461486-39564907
Location of CDK8 gene: 13:26254104-26420982
Location of CEBPA gene: 19:33299934-33302534
Location of CIC gene: 19:42268537-42295797
Location of CTCF gene: 16:67562467-67639177
Location of CYP17A1 gene: 10:102830531-102837472
Location of DDR1 gene: 6:30876421-30900156
Location of DDR2 gene: 1:162631373-162787405
Location of DIS3 gene: 13:72752169-72782096
Location of DNMT3A gene: 2:25227855-25342590
Location of EED gen

In [7]:
missing_gene_lst

['MYLPF',
 'DDX58',
 'H3F3A',
 'MRE11A',
 'FAM123B',
 'FAM46C',
 'C17orf39',
 'MLL',
 'MLL2',
 'MYCL1',
 'WHSC1',
 'WHSC1L1']

In [8]:
synonym_dict = {
    'MYLPF':"MYL11, DA1C, HUMMLC2B, MLC2B, MRLC2, MYLPF",
    'DDX58':"RIGI, DDX58, RIG-I, RIG1, RLR-1, SGMRT2",
    'FAM123B':"AMER1, FAM123B, OSCS, WTX",
    'C17orf39':"GID4, C17orf39, VID2, VID24",
    'H3F3A':"H3-3A, BRYLIB1, H3-3B, H3.3A, H3F3, H3F3A",
    'MLL':"MLLT1, ENL, LTG19, YEATS1",
    'MLL2':"KMT2B, CXXC10, DYT28, HRX2, MLL1B, MLL2, MLL4, MRD68, TRX2, WBP-7, WBP7",
    'MRE11A':"MRE11, ATLD, HNGS1, MRE11A, MRE11B",
    'MYCL1':"MYCL, L-Myc, LMYC, MYCL1, bHLHe38",
    'WHSC1':"NSD2, KMT3F, KMT3G, MMSET, RAUST, REIIBP, TRX5, WHS, WHSC1",
    'WHSC1L1':"NSD3, KMT3F, KMT3G, WHISTLE, WHSC1L1",
    'FAM46C':"TENT5C, FAM46C"
}

In [9]:
# gene_info_query
for gene in synonym_dict:
    found = False
    for synonym in synonym_dict[gene].split(", "):
        try:
            result = gene_info_query(synonym)
            print(gene, '-', synonym)
            if not found:
                meta_gene_info_dict[gene] = result
            found = True
        except Exception as e:
            # print(e)
            continue
    if not found:
        print(gene, "Not Found")

Location of MYL11 gene: 16:30370934-30377991
MYLPF - MYL11
Location of RIGI gene: 9:32455302-32526208
DDX58 - RIGI
Location of AMER1 gene: X:64185117-64205708
FAM123B - AMER1
Location of GID4 gene: 17:18039408-18068405
C17orf39 - GID4
Location of H3-3A gene: 1:226061851-226072019
H3F3A - H3-3A
Location of H3-3B gene: 17:75776434-75785893
H3F3A - H3-3B
Location of MLLT1 gene: 19:6210381-6279975
MLL - MLLT1
Location of KMT2B gene: 19:35717973-35738880
MLL2 - KMT2B
Location of MRE11 gene: 11:94415570-94493885
MRE11A - MRE11
Location of MYCL gene: 1:39895426-39902256
MYCL1 - MYCL
Location of NSD2 gene: 4:1871393-1982207
WHSC1 - NSD2
Location of NSD3 gene: 8:38269704-38382272
WHSC1L1 - NSD3
Location of TENT5C gene: 1:117606048-117628389
FAM46C - TENT5C


In [10]:
meta_gene_info_dict['MLL2'] = gene_info_query('KMT2D')
meta_gene_info_dict['MLL'] = gene_info_query('KMT2A')

Location of KMT2D gene: 12:49018975-49060794
Location of KMT2A gene: 11:118436456-118526832


In [11]:
columns = list(genes_2510)

# define row index names
index_names = drug_response_df.ARXSPAN_ID.values

# create empty dataframe with column names, and row index
data = np.zeros((len(index_names), len(columns))) # initialize all value to be 0
cnv_df = pd.DataFrame(data, columns=columns, index=index_names)
cnv_df.shape

(692, 2510)

In [12]:
cnv_df.head()

,UBB,UBC,UBA52,RPS27A,UBA1,SAE1,UBA2,UBA3,UBA7,UBA6,...,GATA6,C17orf39,INPP4B,KLHL6,MLL,MLL2,MYCL1,WHSC1,WHSC1L1,PRKN
ACH-000001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ACH-000002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ACH-000004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ACH-000006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ACH-000007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
import json

with open("meta_gene_info_dict.json", "w") as outfile:
    json.dump(meta_gene_info_dict, outfile)

In [17]:
check_dict = {}
for cell_line_id in cnv_df.index.values:
    for gene in cnv_df.columns.values:
        if gene not in meta_gene_info_dict:
            check_dict[(gene, cell_line_id)] = "Missing Key"
            print("Need check:", gene)
            continue
        if gene == "TERC":
            cnv_df.loc[cell_line_id, gene] = 0
            continue
        try:
            mapping_dict = {"+":1, "-":-1, "0": 0, "U": 0}
            helper_dict = meta_gene_info_dict[gene]
            temp_data = CCLE_cnv.loc[(CCLE_cnv.Chromosome == helper_dict['chromosome'])].loc[cell_line_id]
            if len(temp_data.shape) == 1:
                adj_status = mapping_dict[temp_data['Status']]
            else:
                s_s = temp_data.loc[temp_data.Start<=helper_dict["start"]].iloc[-1]['Start']
                s_e = temp_data.loc[temp_data.End>=helper_dict["end"]].iloc[0]['End']
                status_list = list(map(lambda x: mapping_dict[x], temp_data.loc[(temp_data.Start >= s_s) & 
                                                                                (temp_data.End <= s_e)].Status.values))
                adj_status = status_list[0]
            cnv_df.loc[cell_line_id, gene] = adj_status
        except Exception as e:
            print(gene, cell_line_id, "Error")
            check_dict[(gene, cell_line_id)] = "Error"

CCL3L3 ACH-000001 Error
CCL3L3 ACH-000002 Error
CCL3L3 ACH-000004 Error
CCL3L3 ACH-000006 Error
CCL3L3 ACH-000007 Error
CCL3L3 ACH-000008 Error
CCL3L3 ACH-000009 Error
CCL3L3 ACH-000012 Error
CCL3L3 ACH-000015 Error
CCL3L3 ACH-000018 Error
CCL3L3 ACH-000019 Error
CCL3L3 ACH-000020 Error
CCL3L3 ACH-000021 Error
CCL3L3 ACH-000023 Error
CCL3L3 ACH-000024 Error
CCL3L3 ACH-000030 Error
CCL3L3 ACH-000035 Error
CCL3L3 ACH-000038 Error
CCL3L3 ACH-000040 Error
CCL3L3 ACH-000042 Error
CCL3L3 ACH-000045 Error
CCL3L3 ACH-000046 Error
CCL3L3 ACH-000048 Error
CCL3L3 ACH-000050 Error
CCL3L3 ACH-000052 Error
CCL3L3 ACH-000053 Error
CCL3L3 ACH-000054 Error
CCL3L3 ACH-000055 Error
CCL3L3 ACH-000056 Error
CCL3L3 ACH-000058 Error
CCL3L3 ACH-000059 Error


KeyboardInterrupt: 

In [18]:
### Check gene CCL3L3
gene = "CCL3L3"
cell_line_id = "ACH-000059"
mapping_dict = {"+":1, "-":-1, "0": 0, "U": 0}
helper_dict = meta_gene_info_dict[gene]
temp_data = CCLE_cnv.loc[(CCLE_cnv.Chromosome == helper_dict['chromosome'])].loc[cell_line_id]
if len(temp_data.shape) == 1:
    adj_status = mapping_dict[temp_data['Status']]
else:
    s_s = temp_data.loc[temp_data.Start<=helper_dict["start"]].iloc[-1]['Start']
    s_e = temp_data.loc[temp_data.End>=helper_dict["end"]].iloc[0]['End']
    status_list = list(map(lambda x: mapping_dict[x], temp_data.loc[(temp_data.Start >= s_s) & 
                                                                    (temp_data.End <= s_e)].Status.values))
    adj_status = status_list[0]

KeyError: 'ACH-000059'

In [20]:
helper_dict

{'start': 36194906, 'end': 36196795, 'chromosome': 'CHR_HSCHR17_10_CTG4'}

In [21]:
meta_gene_info_dict['CCL3L3'] = {'start': 36194906, 'end': 36196795, 'chromosome': '17'} # CHR_HSCHR17_10_CTG4 -> 17

In [23]:
gene = "CCL3L3"
cell_line_id = "ACH-000059"
mapping_dict = {"+":1, "-":-1, "0": 0, "U": 0}
helper_dict = meta_gene_info_dict[gene]
temp_data = CCLE_cnv.loc[(CCLE_cnv.Chromosome == helper_dict['chromosome'])].loc[cell_line_id]
if len(temp_data.shape) == 1:
    adj_status = mapping_dict[temp_data['Status']]
else:
    s_s = temp_data.loc[temp_data.Start<=helper_dict["start"]].iloc[-1]['Start']
    s_e = temp_data.loc[temp_data.End>=helper_dict["end"]].iloc[0]['End']
    status_list = list(map(lambda x: mapping_dict[x], temp_data.loc[(temp_data.Start >= s_s) & 
                                                                    (temp_data.End <= s_e)].Status.values))
    adj_status = status_list[0]
adj_status

0

In [28]:
cell_line_done_dict = {}

In [35]:
check_dict = {}
for cell_line_id in cnv_df.index.values:
    if cell_line_id in cell_line_done_dict:
        continue
    print(len(cell_line_done_dict))
    for gene in cnv_df.columns.values:
        if gene not in meta_gene_info_dict:
            check_dict[(gene, cell_line_id)] = "Missing Key"
            print("Need check:", gene)
            continue
        if gene == "TERC":
            cnv_df.loc[cell_line_id, gene] = 0
            continue
        try:
            mapping_dict = {"+":1, "-":-1, "0": 0, "U": 0}
            helper_dict = meta_gene_info_dict[gene]
            temp_data = CCLE_cnv.loc[(CCLE_cnv.Chromosome == helper_dict['chromosome'])].loc[cell_line_id]
            if len(temp_data.shape) == 1:
                adj_status = mapping_dict[temp_data['Status']]
            else:
                s_s = temp_data.loc[temp_data.Start<=helper_dict["start"]].iloc[-1]['Start']
                s_e = temp_data.loc[temp_data.End>=helper_dict["end"]].iloc[0]['End']
                status_list = list(map(lambda x: mapping_dict[x], temp_data.loc[(temp_data.Start >= s_s) & 
                                                                                (temp_data.End <= s_e)].Status.values))
                adj_status = status_list[0]
            cnv_df.loc[cell_line_id, gene] = adj_status
        except Exception as e:
            print(gene, cell_line_id, "Error")
            check_dict[(gene, cell_line_id)] = "Error"
    cell_line_done_dict[cell_line_id] = "Done"

265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514


In [43]:
count = 0
for i in cnv_df.columns:
    if (cnv_df[i] == 0).all():
        count += 1
        print(i)
print(count)

UBA1
UBE2A
UBE2NL
HUWE1
XIAP
MID1
KLHL13
CUL4B
CACNA1F
FGF16
VEGFD
ARAF
RPS6KA3
RPS6KA6
ELK1
IRAK1
FLNA
DUSP9
IKBKG
ACSL4
GK
RRAGB
LPAR4
FGD1
ARHGEF6
PAK3
DIAPH2
MSN
TMSB4X
BTK
CD40LG
CLDN2
CLDN34
CD99
CD99L2
L1CAM
NLGN3
NLGN4X
CXCR3
WAS
AIFM1
IL3RA
EDA
BMP15
IL2RG
IL9R
CRLF2
CSF2RA
IL13RA1
IL13RA2
EDA2R
PORCN
TBL1X
GPC4
PRICKLE3
FOXO4
ATP2B3
CYSLTR1
GRPR
HTR2C
SLC25A5
SLC25A6
PHKA2
PHKA1
CYBB
TIMP1
PGK1
PDHA1
CCNB3
SMC1A
STAG2
GPR119
CNGA2
GRIA3
ATP1B4
TAF1
TAF7L
TAF9B
COL4A6
COL4A5
PPP2R3B
MTCP1
FHL1
TLR7
TLR8
BRCC3
RPA4
TAB3
AR
ATRX
BCOR
BCORL1
KDM5C
KDM6A
MED12
P2RY8
RBM10
TERC
FAM123B
99


In [40]:
mutation_df_324 = pd.read_csv('../0111/DRPreter-main/Data/mutations_raw_324_final.csv')
mutation_df_324.head()

,DepMap_ID,ABL1,ACVR1B,AKT1,AKT2,AKT3,ALK,ALOX12B,FAM123B,APC,...,VEGFA,VHL,WHSC1,WHSC1L1,WT1,XPO1,XRCC2,ZNF217,ZNF703,TERC
0,ACH-000001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
1,ACH-000002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,ACH-000004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
3,ACH-000006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,ACH-000007,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [41]:
genes_324 = list(mutation_df_324.columns[1:])

In [42]:
for gene in genes_324:
    if gene not in cnv_df.columns:
        print(gene)

PARK2


In [45]:
count = 0
for i in genes_324:
    if i in cnv_df.columns:
        if (cnv_df[i] == 0).all():
            count += 1
            print(i)
print(count)

FAM123B
AR
ARAF
ATRX
BCOR
BCORL1
BTK
KDM5C
KDM6A
MED12
P2RY8
RBM10
STAG2
TERC
14


In [46]:
cnv_df.to_csv("CCLE_seg_cnv_2510.csv")

In [47]:
cnv_df_check = pd.read_csv("CCLE_seg_cnv_2510.csv", index_col=0)
cnv_df_check.head()

,UBB,UBC,UBA52,RPS27A,UBA1,SAE1,UBA2,UBA3,UBA7,UBA6,...,GATA6,C17orf39,INPP4B,KLHL6,MLL,MLL2,MYCL1,WHSC1,WHSC1L1,PRKN
ACH-000001,1.0,0.0,-1.0,0.0,0.0,-1.0,1.0,0.0,0.0,1.0,...,-1.0,1.0,0.0,1.0,-1.0,1.0,-1.0,1.0,1.0,-1.0
ACH-000002,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,-1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
ACH-000004,-1.0,1.0,1.0,1.0,0.0,1.0,0.0,-1.0,1.0,-1.0,...,0.0,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
ACH-000006,-1.0,-1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,...,0.0,-1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0
ACH-000007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
